# Aprendizaje por Diferencia Temporal (TD Learning)

**Tutorial Completo: De la Teoría a la Práctica**

---

## Índice

1. [Introducción a TD Learning](#1-introducción-a-td-learning)
2. [Fundamentos Matemáticos](#2-fundamentos-matemáticos)
3. [Q-Learning](#3-q-learning)
4. [SARSA](#4-sarsa)
5. [Comparación Q-Learning vs SARSA](#5-comparación-q-learning-vs-sarsa)
6. [Cliff Walking Problem](#6-cliff-walking-problem)
7. [Ejercicios](#7-ejercicios)
8. [Conclusiones](#8-conclusiones)

---

## 1. Introducción a TD Learning

### ¿Qué es TD Learning?

**Temporal Difference (TD) Learning** combina ideas de:
- **Dynamic Programming**: Actualizaciones bootstrapping (usar estimaciones futuras)
- **Monte Carlo**: Aprender de experiencia sin modelo

### Características Clave

1. **Model-free**: No requiere conocer $p(s',r|s,a)$
2. **Online**: Aprende después de cada step (no espera fin de episodio)
3. **Bootstrapping**: Usa estimaciones de valores futuros para actualizar
4. **Eficiente**: Combina ventajas de DP y MC

### Aplicaciones

- Juegos (AlphaGo usa variantes de TD)
- Robótica (control motor)
- Optimización de procesos
- Finanzas (evaluación de riesgo)

### Diferencia con Monte Carlo

| Aspecto | Monte Carlo | TD Learning |
|---------|-------------|-------------|
| **Modelo** | No requiere | No requiere |
| **Actualización** | Al final del episodio | Después de cada step |
| **Varianza** | Alta | Baja |
| **Sesgo** | Bajo | Medio |
| **Velocidad** | Lenta | Rápida |

## 2. Fundamentos Matemáticos

### 2.1 Error TD (TD Error)

La clave de TD Learning es el **error TD**:

$$\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t)$$

Donde:
- $R_{t+1}$: Recompensa inmediata
- $\gamma$: Factor de descuento
- $V(S_t)$: Estimación actual del estado
- $V(S_{t+1})$: Estimación del siguiente estado (bootstrap)

### 2.2 Actualización TD

La actualización usa el error TD:

$$V(S_t) \leftarrow V(S_t) + \alpha \cdot \delta_t$$

Donde $\alpha$ es la tasa de aprendizaje.

### 2.3 TD Target vs TD Error

- **TD Target**: $R_{t+1} + \gamma V(S_{t+1})$ (lo que queremos aproximar)
- **TD Error**: Diferencia entre target y estimación actual
- **Actualización**: Mover la estimación hacia el target

### 2.4 Q-Learning vs SARSA

**Q-Learning (Off-policy)**:
$$Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$

Aprende acerca de la **mejor acción posible**, sin importar qué acción tome.

**SARSA (On-policy)**:
$$Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma Q(s',a') - Q(s,a)]$$

Aprende acerca de la **acción que realmente toma**, según su política actual.

### 2.5 Exploración vs Explotación

#### Epsilon-Greedy Policy

$$\pi(a|s) = \begin{cases}
1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|} & \text{si } a = \arg\max_a Q(s,a) \\
\frac{\epsilon}{|\mathcal{A}|} & \text{si } a \neq \arg\max_a Q(s,a)
\end{cases}$$

- **ε pequeño**: Principalmente explotación (usar lo que aprendí)
- **ε grande**: Principalmente exploración (probar nuevas acciones)
- **ε decay**: Disminuir ε con el tiempo (más exploración al principio)

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
import seaborn as sns
from collections import defaultdict
import gymnasium as gym

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

# Configurar reproducibilidad
np.random.seed(42)

print('✓ Importaciones completadas')
print('  - NumPy, Matplotlib, Gymnasium, Seaborn')

## 3. Q-Learning

### 3.1 Algoritmo Q-Learning (Off-Policy)

Q-Learning aprende la **política óptima** mientras explora con una política diferente.

**Algoritmo**:
```
Inicializar Q(s,a) = 0 para todo s,a
Para cada episodio:
    s = estado inicial
    Para cada step:
        a = seleccionar acción con epsilon-greedy
        ejecutar a, observar r, s'
        Q(s,a) <- Q(s,a) + alpha[r + gamma max_a' Q(s',a') - Q(s,a)]
        s = s'
    hasta terminar
```

**Key**: Usa max(a') Q(s',a') en el update (mejor acción posible).

In [ ]:
class QLearningAgent:
    """Agente Q-Learning (off-policy)"""
    
    def __init__(self, n_actions, alpha=0.1, gamma=0.99, epsilon=1.0,
                 epsilon_decay=0.995, epsilon_min=0.01):
        self.n_actions = n_actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.Q = defaultdict(lambda: np.zeros(n_actions))
    
    def get_action(self, state, training=True):
        """Selecciona acción con epsilon-greedy"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        return np.argmax(self.Q[state])
    
    def update(self, state, action, reward, next_state, done):
        """Actualización Q-Learning"""
        current_q = self.Q[state][action]
        
        if done:
            target_q = reward
        else:
            target_q = reward + self.gamma * np.max(self.Q[next_state])
        
        td_error = target_q - current_q
        self.Q[state][action] += self.alpha * td_error
    
    def decay_epsilon(self):
        """Decaer epsilon"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

print('✓ QLearningAgent definido')

### 3.2 Función de Entrenamiento

In [ ]:
def train_agent(env, agent, n_episodes=500, max_steps=100, verbose=True):
    """Entrena agente TD Learning"""
    rewards_history = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        
        for step in range(max_steps):
            action = agent.get_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            agent.update(state, action, reward, next_state, done)
            
            episode_reward += reward
            state = next_state
            
            if done:
                break
        
        agent.decay_epsilon()
        rewards_history.append(episode_reward)
        
        if verbose and (episode + 1) % 100 == 0:
            avg_reward = np.mean(rewards_history[-100:])
            print(f'Ep {episode+1}/{n_episodes} | Reward: {avg_reward:.2f} | epsilon: {agent.epsilon:.3f}')
    
    return rewards_history

print('✓ train_agent definida')

### 3.3 Experimentar con Q-Learning

In [ ]:
# Crear ambiente FrozenLake (determinístico)
env_ql = gym.make('FrozenLake-v1', is_slippery=False)
print('FrozenLake-v1 (determinístico)')
print(f'  Estados: {env_ql.observation_space.n}')
print(f'  Acciones: {env_ql.action_space.n}\n')

# Entrenar Q-Learning
agent_ql = QLearningAgent(
    n_actions=env_ql.action_space.n,
    alpha=0.1,
    gamma=0.99,
    epsilon=1.0,
    epsilon_decay=0.995
)

print('Entrenando Q-Learning...')
rewards_ql = train_agent(env_ql, agent_ql, n_episodes=500, verbose=True)

print(f'\nReward promedio últimos 100 episodios: {np.mean(rewards_ql[-100:]):.3f}')
print(f'Episodios exitosos: {sum([1 for r in rewards_ql[-100:] if r > 0.5])}/100')

env_ql.close()

### 3.4 Visualizar Aprendizaje

In [ ]:
# Gráfico de aprendizaje
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Rewards por episodio
ax1.plot(rewards_ql, alpha=0.5, label='Reward por episodio')

# Promedio móvil
window = 50
moving_avg = np.convolve(rewards_ql, np.ones(window)/window, mode='valid')
ax1.plot(range(window-1, len(rewards_ql)), moving_avg, 'r-', linewidth=2, 
         label=f'Promedio móvil ({window} eps)')

ax1.set_xlabel('Episodio')
ax1.set_ylabel('Reward')
ax1.set_title('Q-Learning: Convergencia')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Histograma de rewards últimos 100
ax2.hist(rewards_ql[-100:], bins=20, edgecolor='black', alpha=0.7)
ax2.set_xlabel('Reward')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Q-Learning: Distribución de Rewards (últimos 100 ep)')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('Q-Learning aprendió exitosamente!')

## 4. SARSA

### 4.1 Algoritmo SARSA (On-Policy)

SARSA aprende sobre la **política que está siguiendo** (on-policy).

**Algoritmo**:
```
Inicializar Q(s,a) = 0 para todo s,a
Para cada episodio:
    s = estado inicial
    a = seleccionar acción con epsilon-greedy desde s
    Para cada step:
        ejecutar a, observar r, s'
        a' = seleccionar acción con epsilon-greedy desde s'
        Q(s,a) <- Q(s,a) + alpha[r + gamma Q(s',a') - Q(s,a)]
        s = s', a = a'
    hasta terminar
```

**Key**: Usa Q(s',a') donde a' es la acción que realmente se tomará (on-policy).

In [ ]:
class SARSAAgent:
    """Agente SARSA (on-policy)"""
    
    def __init__(self, n_actions, alpha=0.1, gamma=0.99, epsilon=1.0,
                 epsilon_decay=0.995, epsilon_min=0.01):
        self.n_actions = n_actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.Q = defaultdict(lambda: np.zeros(n_actions))
    
    def get_action(self, state, training=True):
        """Selecciona acción con epsilon-greedy"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        return np.argmax(self.Q[state])
    
    def update(self, state, action, reward, next_state, next_action, done):
        """Actualización SARSA"""
        current_q = self.Q[state][action]
        
        if done:
            target_q = reward
        else:
            target_q = reward + self.gamma * self.Q[next_state][next_action]
        
        td_error = target_q - current_q
        self.Q[state][action] += self.alpha * td_error
    
    def decay_epsilon(self):
        """Decaer epsilon"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

print('✓ SARSAAgent definido')

### 4.2 Función de Entrenamiento SARSA

In [ ]:
def train_sarsa(env, agent, n_episodes=500, max_steps=100, verbose=True):
    """Entrena agente SARSA"""
    rewards_history = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        action = agent.get_action(state)
        episode_reward = 0
        
        for step in range(max_steps):
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            next_action = agent.get_action(next_state)
            
            agent.update(state, action, reward, next_state, next_action, done)
            
            episode_reward += reward
            state = next_state
            action = next_action
            
            if done:
                break
        
        agent.decay_epsilon()
        rewards_history.append(episode_reward)
        
        if verbose and (episode + 1) % 100 == 0:
            avg_reward = np.mean(rewards_history[-100:])
            print(f'Ep {episode+1}/{n_episodes} | Reward: {avg_reward:.2f} | epsilon: {agent.epsilon:.3f}')
    
    return rewards_history

print('✓ train_sarsa definida')

### 4.3 Experimentar con SARSA

In [ ]:
# Crear ambiente (mismo que para Q-Learning)
env_sarsa = gym.make('FrozenLake-v1', is_slippery=False)

# Entrenar SARSA
agent_sarsa = SARSAAgent(
    n_actions=env_sarsa.action_space.n,
    alpha=0.1,
    gamma=0.99,
    epsilon=1.0,
    epsilon_decay=0.995
)

print('Entrenando SARSA...')
rewards_sarsa = train_sarsa(env_sarsa, agent_sarsa, n_episodes=500, verbose=True)

print(f'\nReward promedio últimos 100 episodios: {np.mean(rewards_sarsa[-100:]):.3f}')
print(f'Episodios exitosos: {sum([1 for r in rewards_sarsa[-100:] if r > 0.5])}/100')

env_sarsa.close()

### 4.4 Visualizar Aprendizaje SARSA

In [ ]:
# Gráfico de aprendizaje SARSA
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Rewards por episodio
ax1.plot(rewards_sarsa, alpha=0.5, label='Reward por episodio')

# Promedio móvil
window = 50
moving_avg = np.convolve(rewards_sarsa, np.ones(window)/window, mode='valid')
ax1.plot(range(window-1, len(rewards_sarsa)), moving_avg, 'g-', linewidth=2,
         label=f'Promedio móvil ({window} eps)')

ax1.set_xlabel('Episodio')
ax1.set_ylabel('Reward')
ax1.set_title('SARSA: Convergencia')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Histograma
ax2.hist(rewards_sarsa[-100:], bins=20, edgecolor='black', alpha=0.7, color='green')
ax2.set_xlabel('Reward')
ax2.set_ylabel('Frecuencia')
ax2.set_title('SARSA: Distribución de Rewards (últimos 100 ep)')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('SARSA aprendió exitosamente!')

## 5. Comparación Q-Learning vs SARSA

### 5.1 Diferencias Teóricas

| Aspecto | Q-Learning | SARSA |
|---------|-----------|-------|
| **Policy** | Off-policy | On-policy |
| **Qué aprende** | Política óptima | Política actual |
| **Target** | max(a') Q(s',a') | Q(s',a') |
| **Riesgo** | Menor | Mayor |
| **Convergencia** | Garantizado | Garantizado |
| **Estabilidad** | Media | Alta |

### 5.2 Cuándo Usar Cada Uno

**Q-Learning**: Cuando seguridad es importante, simulación offline, exploración agresiva

**SARSA**: Cuando el agente debe ser cauteloso, learning online, cuando la exploración tiene riesgo

### 5.3 Comparación Empírica

In [ ]:
# Entrenar múltiples veces para obtener estadísticas
n_runs = 5
n_episodes = 500

print(f'Ejecutando {n_runs} runs de entrenamiento...\n')

all_ql_rewards = []
all_sarsa_rewards = []

for run in range(n_runs):
    # Q-Learning
    env = gym.make('FrozenLake-v1', is_slippery=False)
    agent_ql = QLearningAgent(env.action_space.n, alpha=0.1, gamma=0.99, epsilon=1.0)
    ql_rewards = train_agent(env, agent_ql, n_episodes=n_episodes, verbose=False)
    all_ql_rewards.append(ql_rewards)
    env.close()
    
    # SARSA
    env = gym.make('FrozenLake-v1', is_slippery=False)
    agent_sarsa = SARSAAgent(env.action_space.n, alpha=0.1, gamma=0.99, epsilon=1.0)
    sarsa_rewards = train_sarsa(env, agent_sarsa, n_episodes=n_episodes, verbose=False)
    all_sarsa_rewards.append(sarsa_rewards)
    env.close()
    
    print(f'Run {run+1}/{n_runs} completado')

# Calcular promedios
avg_ql = np.mean(all_ql_rewards, axis=0)
avg_sarsa = np.mean(all_sarsa_rewards, axis=0)

print('\n✓ Entrenamiento completado')

### 5.4 Visualizar Comparación

In [ ]:
# Comparación visual
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Grafica 1: Trayectorias Q-Learning
ax = axes[0, 0]
window = 50
for i in range(n_runs):
    moving_avg_ql = np.convolve(all_ql_rewards[i], np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, n_episodes), moving_avg_ql, alpha=0.3, color='blue')

moving_avg_ql_mean = np.convolve(avg_ql, np.ones(window)/window, mode='valid')
ax.plot(range(window-1, n_episodes), moving_avg_ql_mean, 'b-', linewidth=3, label='Q-Learning (promedio)')
ax.set_xlabel('Episodio')
ax.set_ylabel('Reward')
ax.set_title('Q-Learning: Convergencia')
ax.legend()
ax.grid(True, alpha=0.3)

# Grafica 2: Trayectorias SARSA
ax = axes[0, 1]
for i in range(n_runs):
    moving_avg_sarsa = np.convolve(all_sarsa_rewards[i], np.ones(window)/window, mode='valid')
    ax.plot(range(window-1, n_episodes), moving_avg_sarsa, alpha=0.3, color='green')

moving_avg_sarsa_mean = np.convolve(avg_sarsa, np.ones(window)/window, mode='valid')
ax.plot(range(window-1, n_episodes), moving_avg_sarsa_mean, 'g-', linewidth=3, label='SARSA (promedio)')
ax.set_xlabel('Episodio')
ax.set_ylabel('Reward')
ax.set_title('SARSA: Convergencia')
ax.legend()
ax.grid(True, alpha=0.3)

# Grafica 3: Comparación directa
ax = axes[1, 0]
ax.plot(range(window-1, n_episodes), moving_avg_ql_mean, 'b-', linewidth=2, label='Q-Learning')
ax.plot(range(window-1, n_episodes), moving_avg_sarsa_mean, 'g-', linewidth=2, label='SARSA')
ax.set_xlabel('Episodio')
ax.set_ylabel('Reward (promedio móvil)')
ax.set_title('Comparación: Q-Learning vs SARSA')
ax.legend()
ax.grid(True, alpha=0.3)

# Grafica 4: Performance final
ax = axes[1, 1]
final_ql = np.mean(all_ql_rewards, axis=0)[-100:]
final_sarsa = np.mean(all_sarsa_rewards, axis=0)[-100:]

algorithms = ['Q-Learning', 'SARSA']
means = [np.mean(final_ql), np.mean(final_sarsa)]
stds = [np.std(final_ql), np.std(final_sarsa)]

x = np.arange(len(algorithms))
ax.bar(x, means, yerr=stds, capsize=10, alpha=0.7, color=['blue', 'green'])
ax.set_ylabel('Reward Promedio')
ax.set_title('Performance Final (últimos 100 episodios)')
ax.set_xticks(x)
ax.set_xticklabels(algorithms)
ax.grid(True, alpha=0.3, axis='y')

for i, (mean, std) in enumerate(zip(means, stds)):
    ax.text(i, mean + std + 0.05, f'{mean:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'\nQ-Learning promedio final: {np.mean(final_ql):.3f} ± {np.std(final_ql):.3f}')
print(f'SARSA promedio final: {np.mean(final_sarsa):.3f} ± {np.std(final_sarsa):.3f}')

### 5.5 Conclusiones de la Comparación

In [ ]:
print('='*60)
print('ANÁLISIS COMPARATIVO: Q-LEARNING vs SARSA')
print('='*60)

ql_final = np.mean([np.mean(r[-100:]) for r in all_ql_rewards])
sarsa_final = np.mean([np.mean(r[-100:]) for r in all_sarsa_rewards])

print(f'\nReward Promedio Final:')
print(f'  Q-Learning: {ql_final:.4f}')
print(f'  SARSA:      {sarsa_final:.4f}')
print(f'  Diferencia: {abs(ql_final - sarsa_final):.4f}')

print(f'\nNota: En entornos determinísticos, ambos convergen similar.')
print(f'La diferencia es más notable en entornos estocásticos.')
print('='*60)

## 6. Cliff Walking Problem

### 6.1 Descripción del Problema

El **Cliff Walking** es un entorno clásico donde:
- Grid 4x12 (fila x columna)
- Agente comienza en abajo-izquierda (3,0)
- Meta en abajo-derecha (3,11)
- Un acantilado (cliff) en (3, 1-10) con recompensa -100
- Cada paso normal: recompensa -1

El dilema:
- **Ruta arriesgada**: Cerca del borde (rápida pero -100 si cae)
- **Ruta segura**: Lejos del borde (lenta pero segura)

In [ ]:
# Crear entorno Cliff Walking personalizado
class CliffWalkingEnv:
    def __init__(self):
        self.grid_shape = (4, 12)
        self.start = (3, 0)
        self.goal = (3, 11)
        self.current_pos = self.start
        self.action_names = ['up', 'right', 'down', 'left']
        self.state = self._pos_to_state(self.start)
    
    def _pos_to_state(self, pos):
        return pos[0] * self.grid_shape[1] + pos[1]
    
    def _state_to_pos(self, state):
        return (state // self.grid_shape[1], state % self.grid_shape[1])
    
    def _is_cliff(self, pos):
        return pos[0] == 3 and 1 <= pos[1] <= 10
    
    def reset(self):
        self.current_pos = self.start
        self.state = self._pos_to_state(self.start)
        return self.state, {}
    
    def step(self, action):
        row, col = self.current_pos
        
        if action == 0:  # up
            row = max(0, row - 1)
        elif action == 1:  # right
            col = min(self.grid_shape[1] - 1, col + 1)
        elif action == 2:  # down
            row = min(self.grid_shape[0] - 1, row + 1)
        elif action == 3:  # left
            col = max(0, col - 1)
        
        new_pos = (row, col)
        
        if self._is_cliff(new_pos):
            reward = -100
            new_pos = self.start
            done = True
        elif new_pos == self.goal:
            reward = 0
            done = True
        else:
            reward = -1
            done = False
        
        self.current_pos = new_pos
        self.state = self._pos_to_state(new_pos)
        
        return self.state, reward, done, False, {}

env_cliff = CliffWalkingEnv()
print('✓ Entorno Cliff Walking creado')
print(f'Grid: {env_cliff.grid_shape[0]}x{env_cliff.grid_shape[1]}')
print(f'Total de estados: {env_cliff.grid_shape[0] * env_cliff.grid_shape[1]}')

### 6.2 Entrenar en Cliff Walking

In [ ]:
# Entrenar Q-Learning en Cliff Walking
print('Entrenando Q-Learning en Cliff Walking...')
agent_ql_cliff = QLearningAgent(n_actions=4, alpha=0.1, gamma=0.99, epsilon=0.1)

ql_cliff_rewards = []
for episode in range(500):
    state, _ = env_cliff.reset()
    episode_reward = 0
    
    for step in range(100):
        action = agent_ql_cliff.get_action(state)
        next_state, reward, done, _, _ = env_cliff.step(action)
        agent_ql_cliff.update(state, action, reward, next_state, done)
        episode_reward += reward
        state = next_state
        if done:
            break
    
    agent_ql_cliff.decay_epsilon()
    ql_cliff_rewards.append(episode_reward)
    
    if (episode + 1) % 100 == 0:
        avg = np.mean(ql_cliff_rewards[-100:])
        print(f'Episodio {episode+1}: Avg Reward = {avg:.2f}')

print('\n✓ Q-Learning entrenado')

# Entrenar SARSA en Cliff Walking
print('\nEntrenando SARSA en Cliff Walking...')
agent_sarsa_cliff = SARSAAgent(n_actions=4, alpha=0.1, gamma=0.99, epsilon=0.1)

sarsa_cliff_rewards = []
for episode in range(500):
    state, _ = env_cliff.reset()
    action = agent_sarsa_cliff.get_action(state)
    episode_reward = 0
    
    for step in range(100):
        next_state, reward, done, _, _ = env_cliff.step(action)
        next_action = agent_sarsa_cliff.get_action(next_state)
        agent_sarsa_cliff.update(state, action, reward, next_state, next_action, done)
        episode_reward += reward
        state = next_state
        action = next_action
        if done:
            break
    
    agent_sarsa_cliff.decay_epsilon()
    sarsa_cliff_rewards.append(episode_reward)
    
    if (episode + 1) % 100 == 0:
        avg = np.mean(sarsa_cliff_rewards[-100:])
        print(f'Episodio {episode+1}: Avg Reward = {avg:.2f}')

print('\n✓ SARSA entrenado')

### 6.3 Comparar Resultados

In [ ]:
# Comparar rewards
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

window = 50
moving_avg_ql = np.convolve(ql_cliff_rewards, np.ones(window)/window, mode='valid')
moving_avg_sarsa = np.convolve(sarsa_cliff_rewards, np.ones(window)/window, mode='valid')

axes[0].plot(range(window-1, 500), moving_avg_ql, 'b-', linewidth=2, label='Q-Learning')
axes[0].plot(range(window-1, 500), moving_avg_sarsa, 'g-', linewidth=2, label='SARSA')
axes[0].set_xlabel('Episodio')
axes[0].set_ylabel('Reward')
axes[0].set_title('Cliff Walking: Comparación de Algoritmos')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Performance final
final_ql = np.mean(ql_cliff_rewards[-100:])
final_sarsa = np.mean(sarsa_cliff_rewards[-100:])

algorithms = ['Q-Learning', 'SARSA']
rewards = [final_ql, final_sarsa]
colors = ['blue', 'green']

axes[1].bar(algorithms, rewards, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Reward Promedio')
axes[1].set_title('Performance Final (últimos 100 episodios)')
axes[1].grid(True, alpha=0.3, axis='y')

for i, (algo, reward) in enumerate(zip(algorithms, rewards)):
    axes[1].text(i, reward-5, f'{reward:.1f}', ha='center', fontweight='bold', color='white')

plt.tight_layout()
plt.show()

print(f'\nQ-Learning reward promedio: {final_ql:.2f}')
print(f'SARSA reward promedio: {final_sarsa:.2f}')
print(f'\nObservación: Q-Learning toma ruta arriesgada (óptima).')
print(f'SARSA es más cauteloso (evita el cliff durante entrenamiento).')

## 7. Ejercicios

### Ejercicio 1: Hiperparámetros
Experimenta con diferentes valores de alpha, gamma y epsilon_decay.
¿Cómo afectan la convergencia?

### Ejercicio 2: Expected SARSA
Implementa Expected SARSA que usa la esperanza sobre la política.
¿Es mejor que SARSA?

### Ejercicio 3: Escalabilidad
¿Cómo escalan Q-Learning y SARSA con tamaños mayores del problema?

### Ejercicio 4: Exploración
¿Qué pasa con diferentes políticas de exploración?

### Ejercicio 5: Análisis
Compara convergencia en entornos determinísticos vs estocásticos.

In [ ]:
# EJERCICIO 1: Hiperparámetros
# Experimenta con diferentes alpha, gamma, epsilon_decay
# Tu código aquí...

print('Completa el Ejercicio 1')

In [ ]:
# EJERCICIO 2: Expected SARSA
# Implementa Expected SARSA

class ExpectedSARSAAgent:
    """Implementa Expected SARSA"""
    # Completa esta implementación
    pass

print('Completa el Ejercicio 2')

In [ ]:
# EJERCICIO 3: Escalabilidad
# Prueba con diferentes tamaños de problema
# Tu código aquí...

print('Completa el Ejercicio 3')

In [ ]:
# EJERCICIO 4: Exploración vs Explotación
# Analiza el impacto de diferentes valores de epsilon
# Tu código aquí...

print('Completa el Ejercicio 4')

In [ ]:
# EJERCICIO 5: Análisis Adicional
# Compara comportamiento en entornos determinísticos vs estocásticos
# Tu código aquí...

print('Completa el Ejercicio 5')

## 8. Conclusiones

### Puntos Clave Aprendidos

1. **TD Learning combina DP y MC**:
   - Como DP: usa bootstrapping (estimaciones futuras)
   - Como MC: aprende de experiencia sin modelo

2. **Q-Learning (off-policy)**:
   - Aprende la política óptima
   - Puede explorar agresivamente
   - Mejor para offline learning

3. **SARSA (on-policy)**:
   - Aprende sobre la política que está usando
   - Más cauteloso
   - Mejor cuando la exploración tiene costo

4. **Convergencia garantizada**:
   - Bajo condiciones apropiadas
   - Garantía de convergencia a Q*

### Limitaciones y Extensiones

Limitaciones:
- Requiere discretización de estados
- Lento para espacios grandes
- Puede sobreestimar valores

Extensiones:
- Double Q-Learning: Reduce sobrestimación
- Dueling Q-Learning: Separa valor y ventaja
- Deep Q-Networks (DQN): Usa redes neuronales
- Policy Gradient Methods: Aprende política directamente

### Referencias Clave

1. **Sutton & Barto (2018)**: *Reinforcement Learning: An Introduction*
   - Capítulo 6: Temporal-Difference Learning
   - Capítulo 7: Multi-step Bootstrapping

2. **Watkins & Dayan (1992)**: "Q-learning" (Paper seminal)

3. **Rummery & Niranjan (1994)**: "On-line Q-learning using connectionist systems"

### Próximos Pasos

Después de dominar TD Learning:
1. Aproximación de funciones y escalabilidad
2. Redes neuronales profundas (DQN)
3. Métodos de gradiente de política (Policy Gradient)
4. Actor-Critic methods
5. Aprendizaje por refuerzo modelo-based

---

**¡Felicidades!** Has aprendido los algoritmos fundamentales de TD Learning.